**개요**
* 캐글 - 타이타닉 생존자 예측하기 (https://www.youtube.com/watch?v=aqp_9HV58Ls)에 제시된 방향으로 수행
* Google Drive의 data file을 IO
* 사용 라이브러리: pandas, seaborn


** Preprocessing 방향 **

| 속성 | Data type | Nan 개수 | 비고 | 처리 방향 |
|------|-----------|----------|------|-----------|
| Pclass | numeric | 0 | Major |  |
| Name | alphabet | 0 |  | title 생성 후 drop |
| Sex | female/male | 0 | Major | female/male -> 1/0 |
| Age | numeric | 263 | Major | title group의 중간값으로 채운 뒤, binning, [1.0, 0.2]로 치환 |
| SibSp | numeric | 0 |  |  |
| Parch | numeric | 0 |  |  |
| Ticket | alpha-numeric | 0 |  | Drop |
| Fare | numeric | 1 |  | Pclass group의 중간값으로 채운 뒤, binning, 1.0 -> 0.0으로 치환 (비싼것 -> 싼것) |
| Cabin | alpha-numeric | 1014 |  | 앞자리 alphabet 추출, 0.0 -> 2.8로 치환 (A->T) |
| Embarked | S/C/Q | 2 |  | S로 채운 뒤, [0, 1, 2]로 치환 (S, C, Q) |
| Title |  | 0 | Gender와 Age 단위로 구분 | Name에서 title 추출, [0, 1, 2, 3]으로 치환 (여자 young, 여자 old, 남자 young, 남자 old) |


** Reference **
* 허민석: [kaggle-titanic ](https://github.com/minsuk-heo/kaggle-titanic/blob/master/titanic-solution.ipynb)
* 데이터 사이언스 스쿨: [데이터프레임의 데이터 조작](https://datascienceschool.net/view-notebook/aa62265f02fc429aa636ef343c3b1fda/#%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%94%84%EB%A0%88%EC%9E%84%EC%9D%98-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%A1%B0%EC%9E%91)
* dBRang: [Pandas.DataFrame 조작 - 생성, 추가, 삭제, 조회, 메타 등](http://dbrang.tistory.com/994)
* [Load Google Drive CSV into Pandas DataFrame for Google Colaboratory](http://nali.org/load-google-drive-csv-panda-dataframe-google-colab/)


In [1]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_list = drive.ListFile({'q': "'1IFZC8iktgoQvG9KZwGvA5n34y_KgEZHc' in parents and trashed=false"}).GetList() # 1IFZC8iktgoQvG9KZwGvA5n34y_KgEZHc: google drive's id
for file1 in file_list:
   print('title: %s, id: %s' % (file1['title'], file1['id']))
    
train_downloaded = drive.CreateFile({'id': '15Y5AgDUysjkMKxdHV4HGYwewP7iEsJL_'})
train_downloaded.GetContentFile('train.csv')
test_downloaded = drive.CreateFile({'id': '1YpLMSuyF1zdfyLRN4iP2-m48CIhHy8Sw'})
test_downloaded.GetContentFile('test.csv')

# notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해주는 것
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


sns.set() # setting seaborn default for plots


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.head()

title: Train_preproc.csv, id: 1rFMgT7_aya14AVwLjnAAmoZcJrBlKcMZ
title: Test_preproc.csv, id: 1bpKNF4nCtwyL0zLb6YhnIzC5-YThcpv6
title: test.csv, id: 1YpLMSuyF1zdfyLRN4iP2-m48CIhHy8Sw
title: train.csv, id: 15Y5AgDUysjkMKxdHV4HGYwewP7iEsJL_
title: gender_submission.csv, id: 1oG5nS9G3Lzwv1iM4QYk9Mskv7p5gASxx


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


##** Train, Test 데이터 구성 확인 **
* 데이터 항목, type
* 데이터 개수
* 항목 별 null 값 개수

In [0]:
print(train.shape)
print(test.shape)
print("------------------------------")
train.info()
print("------------------------------")
test.info()

In [0]:
print("------------------------------")
print(train.isnull().sum())
print("------------------------------")
print(test.isnull().sum())

##** 데이터 분석 **

In [0]:
classCount = train['Pclass'].value_counts().sort_index()
classCount.plot.bar()

In [0]:
genderCount = train['Sex'].value_counts()
genderCount.plot.bar()

In [0]:
embarkedCount = train['Embarked'].value_counts()
embarkedCount.plot.bar()

In [0]:
classGender1 = train[train['Pclass'] == 1]['Sex'].value_counts()
classGender2 = train[train['Pclass'] == 2]['Sex'].value_counts()
classGender3 = train[train['Pclass'] == 3]['Sex'].value_counts()
df_classGender = pd.DataFrame([classGender1, classGender2, classGender3])
df_classGender.index = ['Pclass1', 'Pclass2', 'Pclass3']
print(df_classGender)
df_classGender.plot.bar()

**Bar Chart for Categorical Features**
* Pclass
* Sex
* SibSp ( # of siblings and spouse)
* Parch ( # of parents and children)
* Embarked
* Cabin

In [0]:
def bar_chart(feature):
    survived = train[train['Survived']==1][feature].value_counts()
    dead = train[train['Survived']==0][feature].value_counts()
    df = pd.DataFrame([survived,dead])
    df.index = ['Survived','Dead']
    df = df.transpose()
    df.plot.bar()

In [0]:
print("생존/사망자 분포: Pclass, Gender, SibSp, Parch, Embark 별")
bar_chart('Pclass')
bar_chart('Sex')
bar_chart('SibSp')
bar_chart('Parch')
bar_chart('Embarked')

In [0]:
print('생존/사망 별 나이 분포')

ageSurvived = train[train['Survived'] == 1]['Age']
ageDead = train[train['Survived'] == 0]['Age']
df_Age = pd.DataFrame([ageSurvived,ageDead])
df_Age.index = ['Survived','Dead']
df_Age = df_Age.transpose()
df_Age.head()
df_Age.plot.hist(alpha=0.8, colormap='Dark2')

<hr>

** Data 분석 내용**

* Age, Cabin, Fare, Embark에 nan 존재
* 승선 인원
  * Pclass 별: 3등실 >> 1등실 > 2등실
  * Gender 별: male >> female (nearly 2 times)
  * Embark 별: Southampton >>> Cherbourg > Queenstowm
* 각 feature 별 생존/사망 분포
  * Pclass
    * 1등실: 생존자 비율이 더 높다
    * 2등실: 사망자 비율이 조금 더 높다
    * 3등실: 사망지 비율이 앞도적 (3 times)
  * Gender
    * 남자: 사망자 비율이 4배
    * 여자: 생존자 비율이 3배
  * SibSp, Parsh
    * 혼자 승선한 경우, 형제자매나 부모자식과 같이 승선한 경우보다 사망률이 앞도적으로 높다(2 times)
  * Embarked
    * Southampton: 사망률이 높다 (2배)
    * Cherbourg: 생존률이 다소 높다
    * Queenstowm: 사망률이 높다 (2배)

<table border=1 cellspacing=0 align=left>
<tr>
  <td><b>구분</b></td>
  <td><b>인원</b></td>
  <td><b>생존률</b></td>
  <td><b>사망률</b></td>
  <td><b>생존 인원</b></td>
  <td><b>사망 인원</b></td>
</tr>
<tr>
  <td>2등실, 어린이</td>
  <td>24</td>
  <td>100%</td>
  <td>0%</td>
  <td>24</td>
  <td>0</td>
</tr>
<tr>
  <td>1등실, 여성</td>
  <td>144</td>
  <td>97%</td>
  <td>3%</td>
  <td>140</td>
  <td>4</td>
</tr>
<tr>
  <td>승무원, 여성</td>
  <td>23</td>
  <td>87%</td>
  <td>13%</td>
  <td>20</td>
  <td>3</td>
</tr>
<tr>
  <td>2등실, 여성</td>
  <td>93</td>
  <td>86%</td>
  <td>14%</td>
  <td>80</td>
  <td>13</td>
</tr>
<tr>
  <td>1등실, 어린이</td>
  <td>6</td>
  <td>83%</td>
  <td>17%</td>
  <td>5</td>
  <td>1</td>
</tr>
<tr>
  <td>3등실, 여성</td>
  <td>165</td>
  <td>46%</td>
  <td>54%</td>
  <td>76</td>
  <td>89</td>
</tr>
<tr>
  <td>1등실, 남성</td>
  <td>175</td>
  <td>33%</td>
  <td>67%</td>
  <td>57</td>
  <td>118</td>
</tr>
<tr>
  <td>3등실, 어린이</td>
  <td>79</td>
  <td>34%</td>
  <td>66%</td>
  <td>27</td>
  <td>52</td>
</tr>
<tr>
  <td>승무원, 남성</td>
  <td>885</td>
  <td>22%</td>
  <td>78%</td>
  <td>192</td>
  <td>693</td>
</tr>
<tr>
  <td>3등실, 남성</td>
  <td>462</td>
  <td>16%</td>
  <td>84%</td>
  <td>75</td>
  <td>387</td>
</tr>
<tr>
  <td>2등실, 남성</td>
  <td>168</td>
  <td>8%</td>
  <td>92%</td>
  <td>14</td>
  <td>154</td>
</tr>
<tr>
  <td>어린이 총합</td>
  <td>109</td>
  <td>51%</td>
  <td>49%</td>
  <td>56</td>
  <td>53</td>
</tr>
<tr>
  <td>여성 총합</td>
  <td>425</td>
  <td>74%</td>
  <td>26%</td>
  <td>316</td>
  <td>109</td>
</tr>
<tr>
  <td>남성 총합</td>
  <td>1690</td>
  <td>20%</td>
  <td>80%</td>
  <td>338</td>
  <td>1352</td>
</tr>
</table>



##**데이터 가공**

In [2]:
train_test_data = [train, test] # combining train and test dataset

for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

train['Title'].value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Major         2
Mlle          2
Col           2
Mme           1
Jonkheer      1
Countess      1
Lady          1
Ms            1
Capt          1
Sir           1
Don           1
Name: Title, dtype: int64

In [0]:
test['Title'].value_counts()

In [0]:
wholeData = pd.merge(train, test, how='outer')
print(wholeData.shape)
wholeData.groupby("Title")["Age"].describe()

** Title 분석 **
* Title은 모든 사람에 대해서 값을 가지고 있다
* Title Groupping
  * 0: Miss, Mlle, Mme, Mrs, Ms, Countess (Female)
  * 1: Dona, Lady (Female)
  * 2: Master (Male)
  * 3: Mr, Col, Don, Dr, Jonkheer, Major, Rev, Sir, Capt (Male)
  
<table border=1 cellspacing=0 align=left>
<tr>
  <td width=100>Title</td><td width=20>개수</td><td width=100>의미</td>
</tr>
<tr>
  <td>Miss</td><td>260</td><td></td>
</tr>
<tr>
  <td>Mlle</td><td>2</td><td>Miss
</tr>
<tr>
  <td>Lady</td><td>1</td><td>
</tr>
<tr>
  <td>Ms</td><td>2</td><td>
</tr>
<tr>
  <td>Mrs</td><td>197</td><td>
</tr>
<tr>
  <td>Countess</td><td>1</td><td>여자 백작
</tr>
<tr>
  <td>Dona</td><td>1</td><td>귀부인
</tr>
<tr>
  <td>Mme</td><td>1</td><td>영부인
</tr>
<tr>
  <td>Mr</td><td>757</td><td>
</tr>
<tr>
  <td>Master</td><td>61</td><td>
</tr>
<tr>
  <td>Dr</td><td>8</td><td>
</tr>
<tr>
  <td>Rev</td><td>8</td><td>목사 
</tr>
<tr>
  <td>Col</td><td>4</td><td>대령
</tr>
<tr>
  <td>Major</td><td>2</td><td>소령, 시장
</tr>
<tr>
  <td>Capt</td><td>1</td><td>선장
</tr>
<tr>
  <td>Don</td><td>1</td><td>귀인(남자)
</tr>
<tr>
  <td>Sir</td><td>1</td><td>
</tr>
<tr>
  <td>Jonkheer</td><td>1</td><td>
</tr>
<tr>
  <td>합계</td><td>1309</td><td>
</tr>
</table>


**Title Group 별 age 분포**
  * 대체로 mean값과 median값이 일치 --> Age가 Nan인 record에 대해서는 소속 Title의 age 중간값을 대표값으로 사용해도 무방해 보임
  * 단, Mr에 대해서는 대표 나이를 더욱 잘 고를 수 있으면 좋을 것으로 보임
    * max를 제외한 구간에서 연령 분포대가 넓고, 고르게 분포돼어 있는 것으로 보임
    * Title이 Mr인 것 중에 Age가 Nan인 것의 비율: 176 / 581
    * Age 가 nan인 것 중에  Mr이 차지하는 비중: 176 / 263
  * Miss, Mrs, Mr: max 값이 튀는 경향이 있음
  * 특이사항
    * Lady: 1명인데 age가 48임
    * Master: age max 값이 14.5


**Title 별 Age가 nan인 record 개수**
<table border=1 cellspacing=0 align=left>
<tr align=center><td width=80>Title</td><td width=50>Count</td></tr>
<tr align=center><td>Miss</td><td>50</td></tr>
<tr align=center><td>Mrs</td><td>27</td></tr>
<tr align=center><td>Ms</td><td>1</td></tr>
<tr align=center><td>Mr</td><td>176</td></tr>
<tr align=center><td>Master</td><td>8</td></tr>
<tr align=center><td>Dr</td><td>1</td></tr>
</table>


##**Age의 Nan을 무엇으로 채워주면 좋을까**

In [3]:
title_mapping = {"Miss"   : 0, "Mlle" : 0, "Mme": 0, "Mrs" : 0, "Ms"  : 0, "Countess": 0, 
                 "Dona"   : 1, "Lady" : 1, 
                 "Master" : 2, 
                 "Mr"     : 3, "Dr"   : 3, "Rev": 3, "Col": 3, "Major": 3, "Capt" : 3, "Don": 3, "Sir": 3, "Jonkheer": 3 }

for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,3


In [4]:
train["Age"].fillna(train.groupby("Title")["Age"].transform("median"), inplace=True)
test["Age"].fillna(test.groupby("Title")["Age"].transform("median"), inplace=True)

print(train.isnull().sum())
print(test.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Title            0
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Title            0
dtype: int64


In [5]:
for dataset in train_test_data:
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 1.0,
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 26), 'Age'] = 0.8,
    dataset.loc[(dataset['Age'] > 26) & (dataset['Age'] <= 36), 'Age'] = 0.6,
    dataset.loc[(dataset['Age'] > 36) & (dataset['Age'] <= 62), 'Age'] = 0.4,
    dataset.loc[ dataset['Age'] > 62, 'Age'] = 0.2

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,0.8,1,0,A/5 21171,7.2500,NaN,S,3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,0.4,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,0.8,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,0.6,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,0.6,0,0,373450,8.0500,NaN,S,3


In [0]:
bar_chart('Age')

In [6]:
for dataset in train_test_data:
  dataset.drop('Name', axis=1, inplace=True)
  
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,male,0.8,1,0,A/5 21171,7.2500,NaN,S,3
1,2,1,1,female,0.4,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,female,0.8,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,female,0.6,1,0,113803,53.1000,C123,S,0
4,5,0,3,male,0.6,0,0,373450,8.0500,NaN,S,3


In [7]:
gender_mapping = {"male" : 0, "female" : 1}

for dataset in train_test_data:
  dataset['Sex'] = dataset['Sex'].map(gender_mapping)
  
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,0,0.8,1,0,A/5 21171,7.2500,NaN,S,3
1,2,1,1,1,0.4,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,1,0.8,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,1,0.6,1,0,113803,53.1000,C123,S,0
4,5,0,3,0,0.6,0,0,373450,8.0500,NaN,S,3


In [8]:
train["Fare"].fillna(train.groupby("Pclass")["Fare"].transform("median"), inplace=True)
test["Fare"].fillna(test.groupby("Pclass")["Fare"].transform("median"), inplace=True)
train.head()

train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Title            0
dtype: int64

In [0]:
test.isnull().sum()

In [9]:
for dataset in train_test_data:
    dataset.loc[ dataset['Fare'] <= 17, 'Fare'] = 0.0,
    dataset.loc[(dataset['Fare'] > 17) & (dataset['Fare'] <= 30), 'Fare'] = 0.3,
    dataset.loc[(dataset['Fare'] > 30) & (dataset['Fare'] <= 100), 'Fare'] = 0.6,
    dataset.loc[ dataset['Fare'] > 100, 'Fare'] = 1.0
    
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,0,0.8,1,0,A/5 21171,0.0,NaN,S,3
1,2,1,1,1,0.4,1,0,PC 17599,0.6,C85,C,0
2,3,1,3,1,0.8,0,0,STON/O2. 3101282,0.0,NaN,S,0
3,4,1,1,1,0.6,1,0,113803,0.6,C123,S,0
4,5,0,3,0,0.6,0,0,373450,0.0,NaN,S,3


In [10]:
embarked_mapping = {"S" : 0, "C" : 1, "Q" : 2}
  
for dataset in train_test_data:
  dataset['Embarked'] = dataset['Embarked'].fillna('S')
  dataset['Embarked'] = dataset['Embarked'].map(embarked_mapping)
  
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Title            0
dtype: int64

In [0]:
test.isnull().sum()

In [0]:
train.head()

In [11]:
for dataset in train_test_data:
    dataset['Cabin'] = dataset['Cabin'].str[:1]
    
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,0,0.8,1,0,A/5 21171,0.0,NaN,0,3
1,2,1,1,1,0.4,1,0,PC 17599,0.6,C,1,0
2,3,1,3,1,0.8,0,0,STON/O2. 3101282,0.0,NaN,0,0
3,4,1,1,1,0.6,1,0,113803,0.6,C,0,0
4,5,0,3,0,0.6,0,0,373450,0.0,NaN,0,3


In [0]:
Pclass1 = train[train['Pclass']==1]['Cabin'].value_counts()
Pclass2 = train[train['Pclass']==2]['Cabin'].value_counts()
Pclass3 = train[train['Pclass']==3]['Cabin'].value_counts()
df = pd.DataFrame([Pclass1, Pclass2, Pclass3])
df.index = ['1st class','2nd class', '3rd class']
df.plot(kind='bar',stacked=True, figsize=(10,5))

In [12]:
cabin_mapping = {"A": 0, "B": 0.4, "C": 0.8, "D": 1.2, "E": 1.6, "F": 2, "G": 2.4, "T": 2.8}
for dataset in train_test_data:
    dataset['Cabin'] = dataset['Cabin'].map(cabin_mapping)
    
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,0,0.8,1,0,A/5 21171,0.0,NaN,0,3
1,2,1,1,1,0.4,1,0,PC 17599,0.6,0.8,1,0
2,3,1,3,1,0.8,0,0,STON/O2. 3101282,0.0,NaN,0,0
3,4,1,1,1,0.6,1,0,113803,0.6,0.8,0,0
4,5,0,3,0,0.6,0,0,373450,0.0,NaN,0,3


In [13]:
train["Cabin"].fillna(train.groupby("Pclass")["Cabin"].transform("median"), inplace=True)
test["Cabin"].fillna(test.groupby("Pclass")["Cabin"].transform("median"), inplace=True)

train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,0,0.8,1,0,A/5 21171,0.0,2.0,0,3
1,2,1,1,1,0.4,1,0,PC 17599,0.6,0.8,1,0
2,3,1,3,1,0.8,0,0,STON/O2. 3101282,0.0,2.0,0,0
3,4,1,1,1,0.6,1,0,113803,0.6,0.8,0,0
4,5,0,3,0,0.6,0,0,373450,0.0,2.0,0,3


In [0]:
train.isnull().sum()

In [0]:
test.isnull().sum()

In [0]:
train = train.drop("Ticket", axis=1)
test = test.drop("Ticket", axis=1)

In [0]:
from google.colab import files

train.to_csv('Train_preproc.csv', index=False)
test.to_csv('Test_preproc.csv', index=False)

files.download('Train_preproc.csv')
files.download('Test_preproc.csv')